# Atomic models for image simulation with ASE <a id='top'></a>

This notebook introduces the Atomic Simulation Environment ([ASE](https://wiki.fysik.dtu.dk/ase/)) for creating atomic models for image simulation.

ASE is a set of tools and Python modules for setting up, manipulating and visualizing atomic structures, the library is used in conjunction with a large number of atomistic simulation codes, for example [GPAW](https://wiki.fysik.dtu.dk/gpaw/) for running DFT simulations. In this notebook, ASE is introduced in the context of running electron microscopy image simulations with [abTEM](https://abtem.readthedocs.io/en/latest/index.html#). 

### Contents:

1. <a href='#the_atoms_object'> The Atoms object
2. <a href='#importing_structures'> Importing structures from files
3. <a href='#visualization'> Visualization
4. <a href='#manipulating'> Manipulating the atoms
5. <a href='#exporting_structures'> Exporting structures to files
6. <a href='#orthogonal'> Orthogonal and periodic supercells
7. <a href='#nanoparticle'> Example: Nanoparticle on amorphous carbon

### Author:
* 20/05/2023 Jacob Madsen - For the HyperSpy workshop at ePSIC 2023

In [ ]:
%matplotlib inline

import numpy as np
import ase
from ase.visualize import view
import matplotlib.pyplot as plt

import abtem as ab

print("Tested with abTEM v1.0.0beta32. Your current version:", ab.__version__)

##  The `Atoms` object <a id='the_atoms_object'></a>

The `Atoms` object defines a collection of atoms. To define `Atoms` from scratch, we need to specify at least three things:

* atomic positions
* atomic numbers 
* a periodic cell

Here, we create a basic model of the N<sub>2</sub> molecule.

In [ ]:
atoms = ase.Atoms("N2", positions=[(0.0, 0.0, 0.0), (1.0, 0.0, 0.0)], cell=[3, 3, 3])

__Note__: abTEM and ASE uses the same unit conventions, as defined in the `ase.units` module. Thus, electron volts (eV), Ångström (Å), and atomic mass units are defined as 1.0.

We can access the corresponding properties as below.

In [ ]:
atoms.positions

In [ ]:
atoms.numbers

In [ ]:
atoms.cell

The `Atoms` can be modified by directly changing the underlying NumPy arrays. Here we create NO.

In [ ]:
atoms.numbers[0] = 8

We can add an additional N atom to create nitrous oxide.

In [ ]:
atoms += ase.Atoms("N", positions=[(2.0, 0, 0)])

atoms

## Visualization <a id='visualization'></a>

We can visualize the atoms using the Matplotlib backend with abTEM's `show_atoms` function. This function shows a 2D projection of the structure perpendicular to a specified plane.

In [ ]:
ab.show_atoms(atoms);

The default ASE GUI, an interactive 3D viewer, may be started using the `view` function.

In [ ]:
from ase.visualize import view

view(atoms)

## Importing structures from files <a id='importing_structures'></a>

ASE can import all the common atomic structure formats, see a full list [here](https://wiki.fysik.dtu.dk/ase/ase/io/io.html). Below we import a `.cif`-file defining a unit cell of strontium titanate (SrTiO<sub>3</sub>).

In [ ]:
srtio3 = ase.io.read("srtio3.cif")

ab.show_atoms(srtio3)

In [ ]:
view(srtio3)

## Manipulating the atoms <a id='manipulating'></a>
abTEM always assumes that the imaging electrons propagate along the $z$-axis in the direction from _negative to positive_ coordinate values. Hence, to choose the zone axis, we need to manipulate the atoms so they are properly aligned.

ASE has a large number of tools for manipulating imported structures, so we can't cover all of them here. As an example, we will look at the `surface` function, which can be used for creating a periodic surface (aligned with the $z$-axis) with a given set of Miller indices.

Here we orient the strontium titanate structure along the (110)-direction.

In [ ]:
from ase.build import surface

srtio3_110 = surface(srtio3, indices=(1, 1, 0), layers=2, periodic=True)

ab.show_atoms(srtio3_110, plane="xy");

Simulations may require a larger crystal, to repeat the atoms by 2 in $x$, 4 in $y$ and 10 in the $z$-direction, we just multiply the atoms.

In [ ]:
repeated_srtio3 = srtio3_110.copy()

repeated_srtio3 *= (2, 4, 10)

ab.show_atoms(repeated_srtio3, legend=True);

The positions and atomic numbers are just `numpy` arrays and hence can be modified in-place. Here, we create an SrTiO<sub>3</sub>/LaTiO<sub>3</sub> interface by changing the atomic numbers of the Sr atoms with a $y$-coordinate less than $7.5 \ \mathrm{Å}$.

In [ ]:
sto_lto = repeated_srtio3.copy()

mask = sto_lto.symbols == "Sr"

mask = mask * (sto_lto.positions[:, 1] < 7.5)

sto_lto.numbers[mask] = 57

Next, we center the atoms in the cell and add $5 \ \mathrm{Å}$ of vacuum at the entrance and exit surfaces along the $z$-axis.

In [ ]:
sto_lto.center(axis=2, vacuum=5)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(8, 4))
ab.show_atoms(sto_lto, ax=ax1)
ab.show_atoms(sto_lto, ax=ax2, plane="yz");

## Exporting structures to files <a id='exporting_structures'></a>

The structures can also be exported in all the common atomic structure formats. Here, we export the manipulated structure as `.cif`, so we can use it in the next tutorial. 

In [ ]:
from ase.io import write

write("sto_lto.cif", sto_lto)

## Orthogonal and periodic supercells <a id='orthogonal'></a>

The multislice algorithm requires an orthogonal periodic atomic structure as its input. However, taking any arbitrary structure and making it periodic and orthogonal is not always trivial. abTEM has a tool for solving this problem.

To demonstrate the tool, we create a graphene structure with the minimal hexagonal unit cell. 

In [ ]:
graphene = ase.build.graphene()

ab.show_atoms(graphene);

Applying `orthogonalize_cell` we find the smallest orthogonal version of a cell. 

In [ ]:
from abtem.structures import orthogonalize_cell

orthogonal_graphene = orthogonalize_cell(graphene)

ab.show_atoms(orthogonal_graphene);

The problem of creating orthogonal cells is not always as trivial for graphene. For those interested in more advanced uses of the `orthogonalize_cell` function, we have a [tutorial](https://abtem.readthedocs.io/en/latest/tutorials/orthogonal_cells.html) dedicated to the subject.

## Nanoparticle on amorphous carbon <a id='nanoparticle'></a>

We finish with a slightly more advanced example, we create a model of a coppper nanoparticle on amorphous carbon. 

A rough model of the carbon substrate is created by taking a diamond crystal and randomly displacing the atoms. The [`bulk`](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#common-bulk-crystals) function can create common bulk crystal structures. 

In [ ]:
from ase.build import bulk

carbon = bulk("C", cubic=True)

carbon *= (14, 14, 14)

carbon.positions[:] += (
    np.random.randn(len(carbon), 3) * 0.5
)  # displace atoms with a standard deviation of .5 Å

carbon.wrap()

ASE has modules for procedurally generating special structures such as [carbon nanotubes](https://wiki.fysik.dtu.dk/ase/ase/build/build.html#nanotubes) and [nanoparticles](https://wiki.fysik.dtu.dk/ase/ase/cluster/cluster.html). Here, we will use the `Decahedron` function to create a Decahedral gold nanoparticle. The nanoparticle is then rotated 30 degrees around the $x$-axis.

In [ ]:
from ase.cluster import Decahedron

cluster = Decahedron(
    "Cu",
    10,  # Number of atoms on the facets perpendicular to the five fold axis.
    2,  # Number of atoms on the facets parallel to the five fold axis.
    0,  # Depth of re-entrence at the pentagon corners.
)

cluster.rotate("x", 30)

The nanoparticle is moved to the top center of the carbon structure using the `.translate` method.

In [ ]:
center_height = 30

translated_cluster = cluster.copy()

translated_cluster.translate(np.diag(carbon.cell) / 2 + (0, 0, center_height))

The nanoparticle and amourphous substrate is combined in a single model by simply adding them together.

In [ ]:
cluster_on_carbon = carbon + translated_cluster

Then the combined structure is centered along the $z$-axis, and the supercell is adjusted such that there is a vacuum of $4 \ \mathrm{Å}$ on the top and bottom of the model.

In [ ]:
cluster_on_carbon.center(axis=2, vacuum=4)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 4))

ab.show_atoms(cluster_on_carbon, plane="yx", ax=ax1)
ab.show_atoms(cluster_on_carbon, plane="xz", ax=ax2)

Finally, the model is written to disk.

In [ ]:
write("cluster_on_carbon.cfg", cluster_on_carbon)